## TFT
- need to create feature like date and day of week. 

In [11]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
# import plotly.express as px
import numpy as np


In [12]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
nwp_data

<xarray.Dataset> Size: 1TB
Dimensions:    (variable: 14, init_time: 5049, step: 49, latitude: 241,
                longitude: 301)
Coordinates:
  * init_time  (init_time) datetime64[ns] 40kB 2017-01-01 ... 2023-12-31T12:0...
  * latitude   (latitude) float64 2kB 60.0 59.95 59.9 59.85 ... 48.1 48.05 48.0
  * longitude  (longitude) float64 2kB -12.0 -11.95 -11.9 ... 2.9 2.95 3.0
  * step       (step) timedelta64[ns] 392B 00:00:00 01:00:00 ... 2 days 00:00:00
  * variable   (variable) <U5 280B 't2m' 'dswrf' 'dlwrf' ... 'v10' 'u100' 'v100'
Data variables:
    ECMWF_UK   (variable, init_time, step, latitude, longitude) float32 1TB ...

In [13]:
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')
pv_data

<xarray.Dataset> Size: 2GB
Dimensions:   (datetime: 387254)
Coordinates:
  * datetime  (datetime) datetime64[ns] 3MB 2018-01-01T06:05:00 ... 2021-10-2...
Data variables: (12/1311)
    10003     (datetime) float32 2MB ...
    10004     (datetime) float32 2MB ...
    10005     (datetime) float32 2MB ...
    10048     (datetime) float32 2MB ...
    10049     (datetime) float32 2MB ...
    10063     (datetime) float32 2MB ...
    ...        ...
    9870      (datetime) float32 2MB ...
    9871      (datetime) float32 2MB ...
    9902      (datetime) float32 2MB ...
    9903      (datetime) float32 2MB ...
    9960      (datetime) float32 2MB ...
    9989      (datetime) float32 2MB ...

In [14]:

meta_data = pd.read_csv("data_files/metadata.csv")
meta_data.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


In [27]:
def get_48_hour_range(start_datetime):
    end_datetime = start_datetime + pd.Timedelta(hours=47, minutes=59)
    return start_datetime, end_datetime

def select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours):
    selected_datetimes = []
    available_datetimes = list(datetimes)

    for _ in range(num_selections):
        if not available_datetimes:
            break
        random_datetime = np.random.choice(available_datetimes)
        selected_datetimes.append(random_datetime)
        
        # Remove datetimes within the min_gap_hours range
        available_datetimes = [dt for dt in available_datetimes if dt > random_datetime + pd.Timedelta(hours=min_gap_hours)]

    return selected_datetimes

In [28]:

hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)
datetimes = pd.to_datetime(hourly_pv_data['datetime'].values)

data_vars = list(pv_data.data_vars)
pv_sites_id = np.random.choice(data_vars, 25, replace=False)
print(pv_sites_id)

data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : []}

num_selections = 50
min_gap_hours = 48

for ss_id in pv_sites_id:
    selected_datetimes = select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours)
    
    for start_datetime in selected_datetimes:
        start, end = get_48_hour_range(start_datetime)
        selected_data = hourly_pv_data.sel(datetime=slice(start, end))
        
        for dt, power in zip(selected_data['datetime'].values, selected_data[ss_id].values):
            data_dict['ss_id'].append(int(ss_id))
            data_dict['pv_datetime'].append(dt)
            data_dict['generation'].append(power)
                                           



(32193,)
['3280' '26992' '2999' '7551' '3087' '4117' '6971' '6648' '10086' '8856'
 '10692' '5177' '16718' '6597' '27004' '7356' '7049' '2903' '7092' '7410'
 '26982' '26866' '26819' '27008' '7285']
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193
32193


In [29]:


pv_df = pd.DataFrame(data_dict)
print(pv_df.shape)
pv_df = pv_df.dropna(subset={'generation'})

pv_df
print(pv_df.shape)


(7883, 3)
(3614, 3)


In [ ]:
pv_df.to_csv("Try.csv")